In [ ]:
import pandas as pd
!pip install langdetect
!pip install semantic-text-similarity

#Libraries needed 
import glob
import json
import re 
import numpy as np
import copy 
import torch 
import matplotlib.pyplot as plt
from langdetect import detect
from semantic_text_similarity.models import ClinicalBertSimilarity
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/supermind/data.csv")
df2 = pd.read_csv("/content/drive/MyDrive/supermind/term_abb.csv")
df3 = pd.read_csv("/content/drive/MyDrive/supermind/term_def.csv")

In [ ]:
df2.head()
print(df2.shape)

(78, 2)


In [ ]:
df3.head(-5)

,terms,definition1,definition2
0,51% attack,A hypothetical situation where more than half ...,NaN
1,51% attack protection,A protection mechanism implemented by several ...,NaN
2,AFK,Away From Keyboard; used on social media platf...,NaN
3,Airdrop,An event where a blockchain project distribute...,NaN
4,Altcoin,Any cryptocurrency that is an alternative to B...,NaN
...,...,...,...
150,Token/Coin Exchange,"An application to buy, sell and trade cryptocu...",NaN
151,Tokenless Ledger,A ledger that doesn't require a native currenc...,NaN
152,Transactions Per Second (TPS),A measurement of the speed of a blockchain. Th...,NaN
153,Transparency,A primary property of public blockchains where...,NaN


In [ ]:
crypto_words = list(df2['terms'])

In [ ]:
df3.drop('definition2',axis=1,inplace=True)

In [ ]:
df2.rename(columns = {'abbreviations':'definition1'}, inplace = True)


In [ ]:
frames = [df3,df2]
df = pd.concat(frames)
print(df.shape)
df.head(-10)

(238, 2)


,terms,definition1
0,51% attack,A hypothetical situation where more than half ...
1,51% attack protection,A protection mechanism implemented by several ...
2,AFK,Away From Keyboard; used on social media platf...
3,Airdrop,An event where a blockchain project distribute...
4,Altcoin,Any cryptocurrency that is an alternative to B...
...,...,...
63,Millibitcoin (0.001 BTC),mBTC
64,Pump-and-Dump scheme,PnD
65,MicroBitcoin (0.000001 BTC),uBTC
66,Unspent Transaction,UXTO


In [ ]:
df['definition1'] = df['definition1'].apply(lambda x: x.lower())
df['terms'] = df['terms'].apply(lambda x: x.lower())
df['definition1'] = df['definition1'].apply(lambda x: re.sub('[^a-z0-9.\(\)!?\s]','',x))
df['definition1']=  df['definition1'].apply(lambda x: re.sub(r"[\([{})\]]", "", x))
df['terms'] = df['terms'].apply(lambda x: re.sub('[^a-z0-9.\(\)!?\s]','',x))
df['terms'] = df['terms'].apply(lambda x: re.sub(r"[\([{})\]]", "", x))
text_list = list(df['definition1'])+list(df['definition1'])+list(df['terms'])+list(df['terms'])
text_list

['a hypothetical situation where more than half of the computing power on a blockchain network is controlled by one person or group thus allowing them to dictate which transactions are verified. this would allow them to prevent other users from completing confirmed transactions and cause havoc within the system and doublespend coins.',
 'a protection mechanism implemented by several cryptocurrencies that require more than 50 of their total hashing power working together as one entity which would make it difficult for attackers since theyd need even more resources and time or if this threshold is below 100 having an additional safeguard feature where at least 66 must agree with every transaction before sending making them unable to doublespend anything without others noticing until these changes are made on the chain permanently.',
 'away from keyboard used on social media platforms like twitter where users share their trading activity but only want to receive messages while theyre logg

In [ ]:
len(text_list)

952

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
url = "https://coinmarketcap.com/alexandria/glossary"
page = requests.get(url)
soup=BeautifulSoup(page.content,'html.parser')
tags = soup.findAll(attrs={'class':'ehzvDo'})
keywords = []
for tag in tags:
  keywords.append(tag.text)
corpus=[]
for word in keywords:
  if("(" in word):
    text_list.append(word[0:word.find("(")]+word[word.find(")")+1:])
    text_list.append(word[word.find("(")+1:word.find(")")])
  else:
    text_list.append(word)
url = "https://abbreviations.yourdictionary.com/articles/major-cryptocurrency-abbreviations.html"
page = requests.get(url)
soup=BeautifulSoup(page.content,'html.parser')
soup
tags = soup.findAll(attrs={'class':'article-body'})
abb = []
abb_new = []
for tag in tags:
  if(tag.find("ul")):
    curr = tag.find("ul")
    txt = list(curr.descendants)
    for i in range(1,len(txt),2):
      abb.append(str(txt[i]))
# print(abb)
for x in abb:
  abb_new.append(re.split("- | –",x)[0])
  abb_new.extend(abb_new[-1].split("or")[0:2])

abb_new = [x.strip() for x in abb_new]
for x in abb_new:
  if re.search("or",x):
    abb_new.remove(x)
text_list.extend(abb_new)
text_list.extend(abb_new)
print(len(text_list))
print("Tokenizing articles into sentences and words.")
sentences = []
from zipfile import ZipFile
# file_loc = '/root/nltk_data/corpora/wordnet.zip'
# with ZipFile(file_loc, 'r') as z:
#   z.extractall('/root/nltk_data/corpora/')
for index, text in enumerate(text_list):
    sentenceList = sent_tokenize(text.lower())
    for sentence in sentenceList:
        wordList = word_tokenize(sentence)
        sentences.append(wordList)
        sentences.append(wordList)
    print("Processing article "+str(index)+" of "+str(len(text_list)), end="\r")
print("A total of "+str(len(sentences))+" sentences have been tokenized for word2vec training.")
print()

2710
Tokenizing articles into sentences and words.
A total of 5758 sentences have been tokenized for word2vec training.



In [ ]:
print("Training word2vec.")
model = Word2Vec(sentences, size=100, window=2, workers=4)
print("Word count:", len(list(model.wv.vocab)))
model.save("word2vec.model")
print("Finished training and saving word2vec.")

Training word2vec.


Word count: 957
Finished training and saving word2vec.


In [ ]:
model.wv.vocab

{'a': <gensim.models.keyedvectors.Vocab at 0x7f63479ae3d0>,
 'situation': <gensim.models.keyedvectors.Vocab at 0x7f633d47b390>,
 'where': <gensim.models.keyedvectors.Vocab at 0x7f6341a4a590>,
 'more': <gensim.models.keyedvectors.Vocab at 0x7f6344a52f10>,
 'than': <gensim.models.keyedvectors.Vocab at 0x7f6344a528d0>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f6344a52c50>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7f6344a52450>,
 'computing': <gensim.models.keyedvectors.Vocab at 0x7f6344a527d0>,
 'power': <gensim.models.keyedvectors.Vocab at 0x7f6344a52710>,
 'on': <gensim.models.keyedvectors.Vocab at 0x7f6344a52e50>,
 'blockchain': <gensim.models.keyedvectors.Vocab at 0x7f6344a52850>,
 'network': <gensim.models.keyedvectors.Vocab at 0x7f63416e52d0>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7f63416e5150>,
 'by': <gensim.models.keyedvectors.Vocab at 0x7f633b560a90>,
 'one': <gensim.models.keyedvectors.Vocab at 0x7f633bbd1110>,
 'person': <gensim.models.keyedvectors.Vocab at

In [ ]:
print("Loading the pre-trained word2vec model.")
model = Word2Vec.load("word2vec.model")
print("Model loaded.")
print()

Loading the pre-trained word2vec model.
Model loaded.



In [ ]:
word = "crypto"
results = model.wv.most_similar(positive=word, topn=1)
print("Input word:", word)
print("Top "+str(1)+" similar words are:")
for index, word in enumerate(results):
    print(str(index+1)+". "+word[0]+" --- Score: "+str(word[1]))

Input word: crypto
Top 1 similar words are:
1. and --- Score: 0.9998034238815308


# Demo

In [ ]:
sentences = list(df1['content'])
import re
import string
import spacy 
def remove_numbers(text):
    # define the pattern to keep
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, ' ', str(text))
def remove_special_characters(text):
    # define the pattern to keep
    text = remove_numbers(text)
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, ' ', str(text))
sentences = [remove_special_characters(sentence) for sentence in sentences]

nlp=spacy.load("en_core_web_sm")
for sentence in sentences[900:1000]:
    ans=""
    doc=nlp(sentence)
    for token in doc:
      st = str(token)
      st = st.strip()
      # print("{}-{}".format(token,token.pos_))
      if((token.pos_=="NOUN" or token.pos_=="PROPN") and len(st)>2):
       
        try:
          results = model.wv.most_similar(positive=str(token).lower(), topn=1)
          
          if(results[0][1]>0.0):
            ans+='\033[92m' + str(token) + '\033[0m'+" "
          else:
            ans+=str(token)+" "
        except:
          ans+=str(token)+" "
      else:
        ans+=str(token)+" "
    print(ans)

Perhaps you can buy some FTM on Ethereum and bridge that . 
 idk if that is treated as native FTM on Fantom 
Sometimes buy token from cex and withdraw to blockchain . You can buy FTM from Binance for example . 
you can use elk finance to bridge , they just convert some ELK to the native token and deduct it from the amount you bridge 
multichain.org does n't suport ftm , avax asfaik , or most times have no swap liquidity on other side so only stables work best there but gets stuck if you have no native on other side . 

 i can use exchange but the whole point is not to . 

 i ve bumped into this problem several times with new networks and wallets , elk finance looks like it can solve this and supports different chains .   thanks all .   :    
We do support both avax and ftm 
Can confirm it is though it 's not very clear from the UI because " wftm " is a thing in Fantom too 
i meant for example , if i want to go from polygon to avax , there 's only option to do so via usdc pool , and not

In [ ]:

# from bs4 import BeautifulSoup
# import requests
# import re
# url = "https://abbreviations.yourdictionary.com/articles/major-cryptocurrency-abbreviations.html"
# page = requests.get(url)
# soup=BeautifulSoup(page.content,'html.parser')
# soup
# tags = soup.findAll(attrs={'class':'article-body'})
# abb = []
# abb_new = []
# for tag in tags:
#   if(tag.find("ul")):
#     curr = tag.find("ul")
#     txt = list(curr.descendants)
#     for i in range(1,len(txt),2):
#       abb.append(str(txt[i]))
# print(abb)
# for x in abb:
#   abb_new.append(re.split("- | –",x)[0])
#   abb_new.extend(abb_new[-1].split("or")[0:2])

# abb_new = [x.strip() for x in abb_new]
# for x in abb_new:
#   if re.search("or",x):
#     abb_new.remove(x)

#     # for lst in curr.find("li"):
#     #   print(lst)
#     # print()
  
#   # for ele in lst:
#   #   if(ele!=None):
#   #     abb.append(ele.find("li").text)
# # keywords = []
# # for tag in tags:
# #   keywords.append(tag.text)
# # corpus=[]
# # for word in keywords:
# #   if("(" in word):
# #     text_list.append(word[0:word.find("(")]+word[word.find(")")+1:])
# #     text_list.append(word[word.find("(")+1:word.find(")")])
# #   else:
# #     text_list.append(word)